# 第5章 RNN

**前馈**（feedforward）是指网络的传播方向是单向的。

单纯的前馈网络无法充分学习时序数据的性质（模式）。于是，**RNN**（Recurrent Neural Network，循环神经网络）便应运而生。


## 5.1 概率语言模型

作为介绍 RNN 的准备，我们将
首先复习上一章的 word2vec，
然后使用概率描述自然语言相关的现象，
最后介绍从概率视角研究语言的“语言模型”。

### 5.1.1 概率视角下的word2vec

![](../images/图5-1.word2vec的CBOW模型：从上下文预测目标词.PNG)
图5-1.word2vec的CBOW模型：从上下文预测目标词

用数学式来表示“当给定 wt−1 和 wt+1 时目标词是 wt 的概率”：
$$
    P(w_t | w_{t-1}, w_{t+1}) \tag{5.1}
$$

![](../images/图5-2.仅有左侧窗口的上下文.PNG)
图5-2.仅有左侧窗口的上下文

在仅将左侧 2 个单词作为上下文的情况下，CBOW 模型输出的概率如式 (5.2) 所示：
$$
    P(w_t|w_{t−2}, w_{t−1}) \tag{5.2}
$$

CBOW 模型的损失函数可以写成式 (5.3)。式(5.3) 是从交叉熵误差推导出来的结果
$$
   L = -\log P(w_t|w_{t−2}, w_{t−1}) \tag{5.3}
$$

### 5.1.2 语言模型

**语言模型**（language model）给出了单词序列发生的概率。
使用概率来评估一个单词序列发生的可能性，即在多大程度上是自然的单词序列。

使用数学式来表示语言模型。
由 $m$ 个单词 $w_1, \cdots , w_m $构成的句子，将单词按 $w_1, \cdots , w_m$ 的顺序出现的概率记为 $P(w_1, ··· , w_m)$。
因为这个概率是多个事件一起发生的概率，所以称为**联合概率**。

使用后验概率可以将这个联合概率 $P(w_1, \cdots , w_m)$ 分解成如下形式：
$$
    P(w_1, \cdots , w_m) = P(w_m | w_1, \cdots, w_{m-1})P(w_{m-1} | w_1, \cdots, w_{m-2})   \\
        \cdots P(w_3 | w_1, w_2)P(w_2|w_1)P(w_1)    \\
    = \prod_{t=1}^{m} P(w_t | w_1, \cdots, w_{t-1})
    \tag{5.4}
$$

式 (5.4) 的结果可以从概率的**乘法定理**推导出来。

$$
    P(A,B) = P(A|B)P(B) \tag{5.5}
$$

![](../images/图5-3.语言模型中的后验概率.PNG)
图5-3.语言模型中的后验概率

我们的目标是求 $P(w_t|w_1, ··· , w_{t−1})$ 这个概率。如果能计算出这个概率，就能求得语言模型的联合概率 $P(w_1, \cdots , w_m)$。

$P(w_t|w_1, ··· , w_{t−1})$ 表示的模型称为**条件语言模型**（conditional
language model），有时也将其称为语言模型。

### 5.1.3 将CBOW模型用作语言模型

$$
    P(w_1, \cdots , w_m) = \prod_{t=1}^{m} P(w_t|w_1, \cdots , w_{t−1})
    \approx \prod_{t=1}^{m} P(w_t|w_{t-2} , w_{t−1})    \tag{5.8}
$$

![](../images/图5-4.需要较长的上下文的问题示例.PNG)
图5-4.需要较长的上下文的问题示例

CBOW 是 Continuous Bag-Of-Words 的简称。Bag-Of-Words 是“一袋子单词”的意思，这意味着袋子中单词的顺序被忽视了。

![](../images/图5-5.PNG)
图5-5 左图是常规的CBOW模型。右图模型的中间层由上下文的各个单词向量“拼接”而成（图中的输入层是one-hot向量）

我们想要的是考虑了上下文中单词顺序的模型。为此，可以像图 5-5 中
的右图那样，在中间层“拼接”（concatenate）上下文的单词向量。实际上，
“Neural Probabilistic Language Model”中提出的模型就采用了这个方法。
但是，如果采用拼接的方法，权重参数的数量将与上下文大小成比例地增加。

RNN 具有一个机制，那就是无论上下文有多长，都能将上下文信息记住。因此，使
用 RNN 可以处理任意长度的时序数据。

## 5.2 RNN

RNN（Recurrent Neural Network）中的 Recurrent 源自拉丁语，意
思是“反复发生”，可以翻译为“重复发生”“周期性地发生”“循环”，因此
RNN 可以直译为“复发神经网络”或者“循环神经网络”。

### 5.2.1 循环神经网络

RNN 的特征就在于拥有这样一个环路（或回路）。这个环路可以使数据不断循环。
通过数据的循环，RNN 一边记住过去的数据，一边更新到最新的数据。

![](../images/图5-6.拥有环路的RNN层.PNG)
图5-6.拥有环路的RNN层

![](../images/图5-7.把层旋转90度.PNG)
图5-7.把层旋转90度  

### 5.2.2 展开循环

![](../images/图5-8.RNN层的循环的展开.PNG)
图5-8.RNN层的循环的展开

由图 5-8 可以看出，各个时刻的 RNN 层接收传给该层的输入和前一个
RNN 层的输出，然后据此计算当前时刻的输出，此时进行的计算可以用下
式表示：
$   h_t = \tanh (h_{t-1}W_h + x_tW_x + b)   \tag{5.9} $

RNN 有两个权重，分别是将输入 $x$ 转化为输出 h 的权重 $W_x$ 和将前一个 RNN 层的输出转化为当前时刻的输出
的权重 $W_h$。此外，还有偏置 $b$。这里，$h_{t−1}$ 和 $x_t$ 都是行向量。

![](../images/图5-9.展开后的RNN层的画法比较.PNG)
图5-9.展开后的RNN层的画法比较

### 5.2.3 Backpropagation Through Time

![](../images/图5-10.将循环展开后的RNN层的误差反向传播法.PNG)
图5-10.将循环展开后的RNN层的误差反向传播法

因为这里的误差反向传播法是“按时间顺序展开的神经网络的误差
反向传播法”，所以称为 Backpropagation Through Time（**基于时间的反向传播**），简称 BPTT。

### 5.2.4 Truncated BPTT

在处理长时序数据时，通常的做法是将网络连接截成适当的长度。具
体来说，就是将时间轴方向上过长的网络在合适的位置进行截断，从而创建
多个小型网络，然后对截出来的小型网络执行误差反向传播法，这个方法称
为 Truncated BPTT（**截断的 BPTT**）。

在 Truncated BPTT 中，网络连接被截断，但严格地讲，只是网络的
反向传播的连接被截断，正向传播的连接依然被维持。

![](../images/图5-11.在适当位置截断反向传播的连接.PNG)
图5-11.在适当位置截断反向传播的连接

![](../images/图5-12.第1个块的正向传播和反向传播.PNG)
图5-12.第1个块的正向传播和反向传播：因为从后面的时刻传来的梯度被截断，所以误
差反向传播法仅在本块内进行

![](../images/图5-13.第2个块的正向传播和反向传播.PNG)
图5-13.第2个块的正向传播和反向传播

![](../images/图5-14.Truncated%20BPTT的数据处理顺序.PNG)
图5-14.Truncated BPTT的数据处理顺序

### 5.2.5 Truncated BPTT的mini-batch学习

![](../images/图5-15.在进行mini-batch学习时，在各批次（各样本）中平移输入数据的开始位置.PNG)
图5-15.在进行mini-batch学习时，在各批次（各样本）中平移输入数据的开始位置

## 5.3 RNN的实现

![](../images/图5-16.基于RNN的神经网络：在水平方向上长度固定.PNG)
图5-16.基于RNN的神经网络：在水平方向上长度固定

![](../images/图5-17.Time%20RNN层：将展开循环后的层视为一个层.PNG)
图5-17.Time RNN层：将展开循环后的层视为一个层

我们将进行 Time RNN 层中的单步处理的层称为“RNN 层”，将一次处理 T 步的层称为“Time RNN 层”。

### 5.3.1 RNN层的实现

RNN正向传播的数学式；
$   h_t = \tanh (h_{t-1}W_h + x_tW_x + b)   \tag{5.10} $

假设批大小是 N，输入向量的维数是 D，隐藏状态向量的维数是 H，则矩阵的形状检查可以像下面这样进行（图 5-18）

![](../images/图5-18.形状检查：在矩阵乘积中，对应维度的元素数量要一致（这里省略偏置）.PNG)
图5-18.形状检查：在矩阵乘积中，对应维度的元素数量要一致（这里省略偏置）

[common/time_layers.py](../common/time_layers.py)





In [ ]:
import numpy as np


class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

![](../images/图5-19.RNN层的计算图（MatMul节点表示矩阵乘积）.PNG)
图5-19.RNN层的计算图（MatMul节点表示矩阵乘积）

![](../images/图5-20.基于RNN层的计算图的反向传播.PNG)
图5-20.基于RNN层的计算图的反向传播

In [ ]:
# RNN 层的 backward()
def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache

    dt = dh_next * (1 - h_next ** 2)
    db = np.sum(dt, axis=0)
    dWh = np.dot(h_prev.T, dt)
    dh_prev = np.dot(dt, Wh.T)
    dWx = np.dot(x.T, dt)
    dx = np.dot(dt, Wx.T)

    self.grads[0][...] = dWx
    self.grads[1][...] = dWh
    self.grads[2][...] = db

    return dx, dh_prev

### 5.3.2 Time RNN层的实现

Time RNN 层由 T 个 RNN 层构成（T 可以设置为任意值），如图 5-21所示。
![](../images/图5-21.Time%20RNN层和RNN层.PNG)
图5-21.Time RNN层和RNN层

![](../images/图5-22.Time%20RNN层将隐藏状态h保存在成员变量中，以在块之间继承隐藏状态.PNG)
图5-22.Time RNN层将隐藏状态h保存在成员变量中，以在块之间继承隐藏状态


Time RNN 层的实现[common/time_layers.py](../common/time_layers.py)


用 stateful 这个参数来控制是否继承隐藏状态。

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh),
                      np.zeros_like(b)]
        self.layers = None
        self.h, self.dh = None, None
        self.stateful = stateful

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

下正向传播的实现。

In [ ]:
def forward(self, xs):
    Wx, Wh, b = self.params
    N, T, D = xs.shape
    D, H = Wx.shape
    self.layers = []
    hs = np.empty((N, T, H), dtype='f')
    if not self.stateful or self.h is None:
        self.h = np.zeros((N, H), dtype='f')
    for t in range(T):
        layer = RNN(*self.params)
    self.h = layer.forward(xs[:, t, :], self.h)
    hs[:, t, :] = self.h
    self.layers.append(layer)
    return hs

![](../images/图5-23.Time_RNN层的反向传播.PNG)
图5-23.Time_RNN层的反向传播

![](../images/图5-24.第t个RNN层的反向传播.PNG)
图5-24.第t个RNN层的反向传播

反向传播的实现：

In [ ]:
def backward(self, dhs):
    Wx, Wh, b = self.params
    N, T, H = dhs.shape
    D, H = Wx.shape

    dxs = np.empty((N, T, D), dtype='f')
    dh = 0
    grads = [0, 0, 0]
    for t in reversed(range(T)):
        layer = self.layers[t]
        dx, dh = layer.backward(dhs[:, t, :] + dh)  # 求和后的梯度
        dxs[:, t, :] = dx
        for i, grad in enumerate(layer.grads):
            grads[i] += grad

    for i, grad in enumerate(grads):
        self.grads[i][...] = grad
    self.dh = dh

    return dxs

在 Time RNN 层中有多个 RNN 层。另外，这些 RNN 层使用相
同的权重。因此，Time RNN 层的（最终）权重梯度是各个 RNN
层的权重梯度之和。

## 5.4 处理时序数据的层的实现

将基于 RNN 的语言模型称为 **RNNLM**（RNN Language Model，RNN 语言模型）

### 5.4.1 RNNLM的全貌图

![](../images/图5-25.RNNLM的网络图（左图是展开前，右图是展开后）.PNG)
图5-25.RNNLM的网络图（左图是展开前，右图是展开后）

图 5-25 中的第 1 层是 Embedding 层，该层将单词 ID 转化为单词的分布式表示（单词向量）。
然后，这个单词向量被输入到 RNN 层。RNN 层向下一层（上方）输出隐藏状态，同时也向下一时刻的 RNN 层（右侧）输出
隐藏状态。RNN 层向上方输出的隐藏状态经过 Affine 层，传给 Softmax 层。

![](../images/图5-26.处理样本语料库“you%20say%20goodbye%20and%20i%20say%20hello.”的RNNLM的例子.PNG)
图5-26.处理样本语料库“you say goodbye and i say hello.”的RNNLM的例子

### 5.4.2 Time层的实现

![](../images/图5-27.将整体处理时序数据的层实现为Time层.PNG)
图5-27.将整体处理时序数据的层实现为Time层

![](../images/图5-28.将Time_Affine层实现为T个Affine层的集合.PNG)
图5-28.将Time_Affine层实现为T个Affine层的集合

Time Affine 层并不是单纯地使用 T 个Affine 层，而是使用矩阵运算实现了高效的整体处理。
源代码（common/time_layers.py 的 TimeAffine 类）

按照图 5-29 所示的网络结构实现 Time Softmax with Loss 层。

![](../images/图5-29.Time_Softmax_with_Loss层的全貌图.PNG)
图5-29.Time_Softmax_with_Loss层的全貌图

$$
   L = \frac{1}{T}(L_0 + L_1 + \cdots + L_{T_1})    \tag{5.11}
$$

## 5.5 RNNLM的学习和评价

### 5.5.1 RNNLM的实现

将 RNNLM 使用的网络实现为 SimpleRnnlm 类

![](../images/图5-30.SimpleRnnlm的层结构：RNN层的状态在类内部进行管理.PNG)
图5-30.SimpleRnnlm的层结构：RNN层的状态在类内部进行管理

[SimpleRnnlm 类](../ch05/simple_rnnlm.py)

RNN 层和 Affine 层使用了“Xavier 初始值”。

![](../images/图5-31.Xavier初始值.PNG)
图5-31.Xavier初始值

### 5.5.2 语言模型的评价

语言模型基于给定的已经出现的单词（信息）输出将要出现的单词的概率分布。
**困惑度**（perplexity）常被用作评价语言模型的预测性能的指标。

困惑度表示“概率的倒数”（这个解释在数据量为 1 时严格一致）。

![](../images/图5-32.当输入单词you时，模型输出下一个出现的单词的概率分布.PNG)
图5-32.当输入单词you时，模型输出下一个出现的单词的概率分布

它们可以解释为“分叉度”。
所谓分叉度，是指下一个可以选择的选项的数量（下一个可能出现的单词的候选个数）。

在输入数据为多个的情况下，结果会怎样呢？我们可以根据下面的式子进行计算。
$$
    L = - \frac{1}{N} \sum_n \sum_k t_{nk} \log y_{nk}  \tag{5.12}  \\
    困惑度=e^L \tag{5.13}
$$

假设数据量为 $N$ 个。$t_n$ 是 one-hot 向量形式的正确解标签，$t_{nk}$ 表
示第 $n$ 个数据的第 $k$ 个值，$y_{nk}$ 表示概率分布（神经网络中的 Softmax 的
输出）。顺便说一下，$L$ 是神经网络的损失，和式 (1.8) 完全相同，使用这个
$L$ 计算出的 $e^L$ 就是困惑度。

在信息论领域，困惑度也称为“平均分叉度”。这可以解释为，数据
量为 1 时的分叉度是数据量为 N 时的分叉度的平均值。

### 5.5.3 RNNLM的学习代码

[ch05/train_custom_loop.py](../ch05/train_custom_loop.py)

![](../images/图5-33.困惑度的演变.PNG)
图5-33.困惑度的演变

### 5.5.4 RNNLM的Trainer类

用于学习 RNNLM 的 RnnlmTrainer 类，其内部封装了刚才的 RNNLM 的学习。将刚才的学习代码重构为 RnnlmTrainer 类
[ch05/train.py](../ch05/train.py)

RnnlmTrainer 类的内部将执行上一节进行的一系列操作，具体如下所示：
* 按顺序生成 mini-batch
* 调用模型的正向传播和反向传播
* 使用优化器更新权重
* 评价困惑度

## 5.6 小结

语言模型给单词序列赋概率值。特别地，条件语言模型从已经出现的单词序列计算下一个将要出现的单词的概
率。通过构成利用了 RNN 的神经网络，理论上无论多么长的时序数据，都
可以将它的重要信息记录在 RNN 的隐藏状态中。但是，在实际问题中，这
样一来，许多情况下学习将无法顺利进行。下一章我们将指出 RNN 存在的
问题，并研究替代 RNN 的 LSTM 层或 GRU 层。这些层在处理时序数据方
面非常重要，被广泛用于前沿研究。

## 本章所学的内容

* RNN 具有环路，因此可以在内部记忆隐藏状态
* 通过展开 RNN 的循环，可以将其解释为多个 RNN 层连接起来的神
经网络，可以通过常规的误差反向传播法进行学习（= BPTT）
* 在学习长时序数据时，要生成长度适中的数据块，进行以块为单位
的 BPTT 学习（= Truncated BPTT）
* Truncated BPTT 只截断反向传播的连接
* 在 Truncated BPTT 中，为了维持正向传播的连接，需要按顺序输
入数据
* 语言模型将单词序列解释为概率
* 理论上，使用 RNN 层的条件语言模型可以记忆所有已出现单词的信息